## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import the API key.
import sys  
sys.path.insert(0, '../')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Marigot,MF,18.0667,-63.0833,86.22,66,40,12.66,scattered clouds
1,1,Yuksekova,TR,37.5736,44.2872,70.43,44,0,5.39,clear sky
2,2,Barrow,US,71.2906,-156.7887,29.52,85,100,10.18,overcast clouds
3,3,Longyearbyen,SJ,78.2186,15.6401,35.44,69,75,8.05,broken clouds
4,4,Hambantota,LK,6.1241,81.1185,81.12,81,81,20.85,broken clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Yuksekova,TR,37.5736,44.2872,70.43,44,0,5.39,clear sky
13,13,Bengkulu,ID,-3.8004,102.2655,77.14,91,100,4.45,overcast clouds
16,16,Rikitea,PF,-23.1203,-134.9692,74.17,66,0,13.40,clear sky
17,17,Ponta Do Sol,PT,32.6667,-17.1000,72.12,73,59,1.99,broken clouds
24,24,Kapaa,US,22.0752,-159.3190,75.18,100,62,2.42,broken clouds
25,25,Buckie,GB,57.6757,-2.9624,73.11,64,41,19.46,scattered clouds
32,32,Rocha,UY,-34.4833,-54.3333,61.90,53,24,18.79,few clouds
37,37,Quatre Cocos,MU,-20.2078,57.7625,73.51,73,1,10.33,clear sky
39,39,Koraput,IN,18.8167,82.7167,78.21,65,94,9.44,moderate rain
40,40,Ribeira Grande,PT,38.5167,-28.7000,64.80,72,75,10.36,broken clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, Weather Description and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yuksekova,TR,70.43,clear sky,37.5736,44.2872,
13,Bengkulu,ID,77.14,overcast clouds,-3.8004,102.2655,
16,Rikitea,PF,74.17,clear sky,-23.1203,-134.9692,
17,Ponta Do Sol,PT,72.12,broken clouds,32.6667,-17.1000,
24,Kapaa,US,75.18,broken clouds,22.0752,-159.3190,
25,Buckie,GB,73.11,scattered clouds,57.6757,-2.9624,
32,Rocha,UY,61.90,few clouds,-34.4833,-54.3333,
37,Quatre Cocos,MU,73.51,clear sky,-20.2078,57.7625,
39,Koraput,IN,78.21,moderate rain,18.8167,82.7167,
40,Ribeira Grande,PT,64.80,broken clouds,38.5167,-28.7000,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary
    # formatting is specific to Google API Call.
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_name = hotels["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"Added {hotel_name} to hotel_df...")
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Added Hotel Grand Yılmaz to hotel_df...
Added Grage Hotel Bengkulu to hotel_df...
Added Pension Maro'i to hotel_df...
Added Hotel do Campo to hotel_df...
Added Sheraton Kauai Coconut Beach Resort to hotel_df...
Added Oran House to hotel_df...
Added Beleza Pura B&B to hotel_df...
Added Emeraude Beach Attitude Hotel to hotel_df...
Added Raj Residency to hotel_df...
Added Quinta da Meia Eira to hotel_df...
Added Villa Enata to hotel_df...
Added Couvant des soeurs catholiques to hotel_df...
Added Finch Bay Galapagos Hotel to hotel_df...
Added Hilton Garden Inn Chicago North Shore/Evanston to hotel_df...
Added Shandrani Beachcomber Resort & Spa to hotel_df...
Added Athabasca Lodge Motel to hotel_df...
Added The Halyards Hotel to hotel_df...
Added Oceano to hotel_df...
Added Hilda Apartman to hotel_df...
Added Blue Bay Lodge to hotel_df...
Added Leumerus Bungalows to hotel_df...
Added Torch Hotel to hotel_df...
Added Hotel América to hotel_df...
Added 7Days Inn Xingtai South Yucai Road to ho

Added Anil Hotel to hotel_df...
Added Gostinitsa to hotel_df...
Added "U Tat'yany" to hotel_df...
Added Screensaver to hotel_df...
Hotel not found... skipping.
Added Saruna Wellness - Hotel Park Belgrade to hotel_df...
Added Grand Hotel President to hotel_df...
Added Tobin Lake Motel to hotel_df...
Added Hôtel LAVOUMA suite to hotel_df...
Hotel not found... skipping.
Added Yos ega cv to hotel_df...
Added Hotel San Rafael to hotel_df...
Added Rtsm Shetpe Analog to hotel_df...
Added Gezhouba Hotel to hotel_df...
Added Stonewall Inn to hotel_df...
Added Kipot to hotel_df...
Added Shanti Maurice Resort & Spa to hotel_df...
Added Maringe Lagoon Lodge to hotel_df...
Added Casa Alencar to hotel_df...
Added Wood River Inn & Suites to hotel_df...
Added Brahmagiri to hotel_df...
Added Days Inn by Wyndham Arlington Six Flags/AT&T Stadium to hotel_df...
Added Hotel Alex Johnson Rapid City, Curio Collection by Hilton to hotel_df...
Added Camp Pālehua to hotel_df...
Added Bonthe Holiday Village to h

In [26]:
# 7. Drop the rows where there is no Hotel Name.
print(f"hotel_df has {len(hotel_df)} rows before removing no-name hotels")
for index, row in hotel_df.iterrows():
    if(row["Hotel Name"] == ""):
        print(f"Removing row at index {index}")
        hotel_df = hotel_df.drop(index)
        
print(f"hotel_df has {len(hotel_df)} rows after removing no-name hotels")

hotel_df has 270 rows before removing no-name hotels
hotel_df has 270 rows after removing no-name hotels


In [27]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Yuksekova,TR,70.43,clear sky,37.5736,44.2872,Hotel Grand Yılmaz
13,Bengkulu,ID,77.14,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
16,Rikitea,PF,74.17,clear sky,-23.1203,-134.9692,Pension Maro'i
17,Ponta Do Sol,PT,72.12,broken clouds,32.6667,-17.1000,Hotel do Campo
24,Kapaa,US,75.18,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
25,Buckie,GB,73.11,scattered clouds,57.6757,-2.9624,Oran House
32,Rocha,UY,61.90,few clouds,-34.4833,-54.3333,Beleza Pura B&B
37,Quatre Cocos,MU,73.51,clear sky,-20.2078,57.7625,Emeraude Beach Attitude Hotel
39,Koraput,IN,78.21,moderate rain,18.8167,82.7167,Raj Residency
40,Ribeira Grande,PT,64.80,broken clouds,38.5167,-28.7000,Quinta da Meia Eira


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))